In [ ]:
import networkx as nx
import pandas as pd
#from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
#from collections import Counter
#import seaborn as sns
#import sys
#import os
from Granovetter_experiment_results import (
    #function draws treshold values for each node in G, 
    #mode determines how treshold values are generated
    #each node samples tershold from specified distribution
    #phi mean and phistd control mean and stdv of distribution
    #inputs = G, mode, phi_mean, phi_std
    draw_thresholds, 
    #simulates treshold dynamics on graph G
    #input = G, list of seeds, dict of tresholds, max_nr_steps 
    #returns dictionary with time series of adopted nodes and numpy array
    # of adopted nodes over time (history)
    threshold_dynamics,
    #input = adopted nodes
    final_adoption_fraction,
    #function to calculate the time to reach a target fraction of adopted nodes
    #inputs = history, target=0.5)
    time_to_fraction,
    #random seeds
    random_seeds,
    #select top-B nodes with highest degree in graph G
    high_degree_seeds,
    #does same as for degree, uses a list comprehension and a lambda function
    #lambda function takes tuple x = (node,betweenness_value) and returns the second
    # value b which is the betweenness centrality(x[1]) in descending order
    # [:B] takes the first B entries from the sorted list, which are the nodes with the highest betweenness centrality.
    #inputs = G, B
    betweenness_seeds,
    #this funciton seeds the network with B nodes with highest core centrality nunmber,
    # core nr is nr of nodes that are connected to all its neighbors
    # for given graph G, the core number of a node v is the largest value k such that v is in a k-core.
    #other words: tells you how deeply embedded a node is within the network's core structure.
    #higher core number means node is more central to the network's connectivity
    #inputs = G, B
    kcore_seeds,
    #estimates the spread of the treshold dynamics on graph G
    #it runs the treshold dynamics R times, with different random seeds and tressholds
    #returns the average final adoption fraction
    #input = G, seeds, B, thresholds, R=5, max_steps =100
    estimate_spread,
    
    run_experiments
) 
Random_seed = 42
random.seed(Random_seed)
np.random.seed(Random_seed)


In [7]:
#store nodes and edges data
def load_data(nodes, edges):

    nodes = pd.read_csv(nodes)
    edges = pd.read_csv(
        edges,
        header=None,
        names=["source_new_id", "target_new_id"],
    )
    # merge to get page names
    df_merged = (
        edges
        .merge(nodes[['new_id', 'name']], left_on='source_new_id', right_on='new_id')
        .rename(columns={'name': 'page_name_1'})
        .drop('new_id', axis=1)
        .merge(nodes[['new_id', 'name']], left_on='target_new_id', right_on='new_id')
        .rename(columns={'name': 'page_name_2'})
        .drop('new_id', axis=1)
        .rename(columns={
            'source_new_id': 'page_id_1',
            'target_new_id': 'page_id_2'
        })
    )
    print(nodes.size)
    print(edges.size)
    return df_merged, nodes, edges

In [9]:
#threadpullexecuter
df_merged = load_data("fb-pages-public-figure.nodes", "fb-pages-public-figure.edges")
df_merged

34677
134228


(       page_id_1  page_id_2    page_name_1  \
 0              0       1490         LubaTV   
 1              0       2742         LubaTV   
 2              0       7265         LubaTV   
 3              0       8076         LubaTV   
 4              0       8844         LubaTV   
 ...          ...        ...            ...   
 67094      11441      11479  David Kempton   
 67095      10900      11486  David Grutman   
 67096      11465      11528    A'N'D Lucia   
 67097      11468      11513      Joe Rogan   
 67098      11479      11527   John McVeigh   
 
                                 page_name_2  
 0                           Erick Krominski  
 1                              Kim RosaCuca  
 2                                Hugo Gloss  
 3                                  CutiePie  
 4                              Jayde Pierce  
 ...                                     ...  
 67094                          John McVeigh  
 67095                     MACK ON THE RADIO  
 67096     

In [ ]:
#Granovetterstyle threshold dynamics
#idea is you have 2 states, 0 and 1. based on when you update node, 
# based on neighbors, if two nodes are from the opposite category, 
# next time step it will stay same if treshold is met, 
# it will change if treshold is not met?
#normalized
#
# explore how the structure of your network affects the spread of behaviour or information under
#different seeding strategies.



#implement 1
#either treshold will be the same for all nodes 
#or treshold will be different for different nodes, draf from a distribution
#

#compare seeding strategies:
#investigate different seeding strategies such as random seeding nodes, 
# targeting high-degree nodes, or targeting nodes based on their 
# centrality measures (is that same as highest-betweenness nodes?).
#seeding strategies (random, high-degree, high-betweenness, k-core)
seeding_strategies = ['random', 'high-degree', 'high-betweenness', 'k-core']


#for each strategy, record and compare:
#- final fraction of active nodes (so state 1?)
# nr of time steps until steady state is reached (or 100% adoption)
# shape of the adoption curve over time


#choose and document:
#Network used (from assignment 2, what is this?)
# nr of seeds (1%, 5% )?
nr_of_seeds = [0.01, 0.05, 0.1]
#treshold values (fixed or distributed)
threshold_distributions = {'beta','fixed', 'uniform', 'normal'}
#number of simulation repititions (30?) to reduce stochastic effects

#with notebook in for loop add something that shows how far in the process you are (how many percent done)

In [ ]:
# df_results = run_experiments(
#     N=N,
#     k_avg=k_avg,
#     ensembles=ENSEMBLES,
#     runs_per_setting=RUNS_PER_SETTING,
#     threshold_mode=THRESHOLD_MODE,
#     phi_mean=PHI_MEAN,
#     phi_std=PHI_STD,
#     B=B,
#     max_steps=MAX_STEPS,
#     greedy_R=GREEDY_R
# )
# df_results.head()

In [ ]:
#B_initial = []
#N = 300 (what is N?)
ensembles = [5,10,15]
runs_per_simulationrun = [10,20,30]
max_steps = [100,200,300]

